<a href="https://colab.research.google.com/github/kavidu-dilhara/Google-Drive-Tools/blob/main/Tools/Google_Drive_Tools.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# <center><b>Google Drive Tools
#### <center>Make Google Drive easier with Google Colab

<center><img src="https://i.ibb.co/GRLrLFw/Photo-1265003955.jpg" alt="Google Drive Tools" width="500"></center>


<font color='Red'><h1>Any Direct File To Google Drive Downloader ⬇️</font>

$\color{red}{\text{Important Note}}$: Enter Only Direct File Link

In [ ]:
#@markdown Mount Google Drive ☁
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
#@markdown  <b><h5>Download Singal URL File To Google Drive</h5></b>
import os
DOWNLOAD_URL = "" #@param {type:"string"}
DOWNLOAD_PATH = "/content/drive/" #@param {type:"string"}
os.chdir(DOWNLOAD_PATH)
!wget -c $DOWNLOAD_URL  --no-check-certificate

$\color{red}{\text{🟢Put your all direct link to text file}}$

In [ ]:
#@markdown  <b><h5>Download Multiple URL File To Google Drive</h5></b>
import os
URL_LIST_PATH = "/content/link.txt" #@param {type:"string"}
DOWNLOAD_PATH = "/content/drive/MyDrive" #@param {type:"string"}
os.chdir(DOWNLOAD_PATH)
!wget -i $URL_LIST_PATH  --no-check-certificate


<font color='red'><h1>This notebook allows for downloading all videos on a Youtube channel to your Google Drive</font>
####You need to have Youtube API key before using this. This notebook use youtube-dl library for downloading the videos.

In [ ]:
#@markdown ### Mount Google Drive<br><h5>To stream files we need to mount Google Drive.
import os
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
#@markdown #Install the required library
!pip install youtube_dl

# Get all video links from the channel
Change api_key to your own Youtube API key, and channel_id to the ID of the channel you wish to download

In [ ]:
import json
from urllib.request import urlopen

api_key = ''
channel_id = ""

def get_all_video_in_channel():
    base_video_url = 'https://www.youtube.com/watch?v='
    base_search_url = 'https://www.googleapis.com/youtube/v3/search?'

    first_url = base_search_url + 'key={}&channelId={}&part=snippet,id&order=date&maxResults=25'.format(api_key, channel_id)
    video_links = []
    url = first_url
    while True:
        inp = urlopen(url)
        resp = json.load(inp)

        for i in resp['items']:
            channel_name = resp['items']
            if i['id']['kind'] == "youtube#video":
                video_links.append(base_video_url + i['id']['videoId'])
        try:
            next_page_token = resp['nextPageToken']
            url = first_url + '&pageToken={}'.format(next_page_token)
        except:
            break
    return video_links


links = get_all_video_in_channel()
for link in links:
  print(link)
channel_api = "https://www.googleapis.com/youtube/v3/channels?part=snippet&id={}&key={}".format(channel_id, api_key)
cname = urlopen(channel_api)
respon = json.load(cname)
channel_name = respon["items"][0]["snippet"]["title"]
print("Channel name: " + channel_name)
print("Total videos found: " + str(len(links)))

In [ ]:
#@markdown # Change the working directory <h5>This will change the working directory to the Google drive folder, by default this will make new folder called "Youtube Downloads" and place the downloaded videos here. You can change it to whatever you want.

os.chdir('gdrive/MyDrive')
!mkdir -p "Youtube Downloads"
os.chdir("Youtube Downloads")
print("Current directory: " + os.getcwd())

In [ ]:
#@markdown # Download the videos using youtube-dl
from future import unicode_literals
import youtube_dl

ydl_opts = {}
with youtube_dl.YoutubeDL(ydl_opts) as ydl:
    ydl.download(links)

<font color='Red'><h1>Torrent To Google Drive Downloader</font>

Important Note: To get more disk space:
> Go to Runtime -> Change Runtime and give GPU as the Hardware Accelerator.  You will get around 384GB to download any torrent you want.

In [ ]:
#@markdown ### Mount Google Drive<br><h5>To stream files we need to mount Google Drive.

from google.colab import drive

drive.mount("/content/drive")

In [ ]:
#@markdown ### Install libtorrent and Initialize Session
!apt install python3-libtorrent

import libtorrent as lt

ses = lt.session()
ses.listen_on(6881, 6891)
downloads = []

In [ ]:
#@markdown ### Add From Torrent File<br> <h5>You can run this cell to add more files as many times as you want
from google.colab import files
DOWNLOAD_PATH = "/content/drive/MyDrive" #@param {type:"string"}
source = files.upload()
params = {
    "save_path": "$DOWNLOAD_PATH",
    "ti": lt.torrent_info(list(source.keys())[0]),
}
downloads.append(ses.add_torrent(params))

In [ ]:
#@markdown ### Add From Magnet Link <br><h5>You can run this cell to add more files as many times as you want
DOWNLOAD_PATH = "/content/drive/MyDrive" #@param {type:"string"}
params = {"save_path": "$DOWNLOAD_PATH"}

while True:
    magnet_link = input("Enter Magnet Link Or Type Exit: ")
    if magnet_link.lower() == "exit":
        break
    downloads.append(
        lt.add_magnet_uri(ses, magnet_link, params)
    )

In [ ]:
#@markdown ### Start Download 
import time
from IPython.display import display
import ipywidgets as widgets

state_str = [
    "queued",
    "checking",
    "downloading metadata",
    "downloading",
    "finished",
    "seeding",
    "allocating",
    "checking fastresume",
]

layout = widgets.Layout(width="auto")
style = {"description_width": "initial"}
download_bars = [
lk

lk👇### Add From Torrent File
You can run this cell to add more files as many times as you want    widgets.FloatSlider(
        step=0.01, disabled=True, layout=layout, style=style
    )
    for _ in downloads
]
display(*download_bars)

while downloads:
    next_shift = 0
    for index, download in enumerate(downloads[:]):
        bar = download_bars[index + next_shift]
        if not download.is_seed():
            s = download.status()

            bar.description = " ".join(
                [
                    download.name(),
                    str(s.download_rate / 1000),
                    "kB/s",
                    state_str[s.state],
                ]
            )
            bar.value = s.progress * 100
        else:
            next_shift -= 1
            ses.remove_torrent(download)
            downloads.remove(download)
            bar.close() # Seems to be not working in Colab (see https://github.com/googlecolab/colabtools/issues/726#issue-486731758)
            download_bars.remove(bar)
            print(download.name(), "complete")
    time.sleep(1)

<font color='red'><h1>Google Drive Streamable Link Generator for Media Files</font>
* Make sure your file is accessable by anyone with the link

* It's helpful to have a publicly shared folder when creating links, then you won't have to edit the Share setting for each individual file.
* This script should work with every type of URL Google Drive can generate. As long as the File ID is there, anything else shouldn't matter.

In [ ]:
#@title ⬅️ ឵឵Click Here to Generate Streamable Links
import re

shared_file_link = "" #@param {type:"string"}
result = re.search("[\w-]{33}", shared_file_link)
your_api_key = "" #@param {type:"string"}
drive_file_id = result.group(0)

beg_url = "https://www.googleapis.com/drive/v3/files/"
end_url = "/?key=" + your_api_key + "&alt=media"

print(beg_url + drive_file_id + end_url)


<font color='red'><h1>Download Spotify Playlist To Google Drive using spotdl</font>

In [ ]:
#@markdown  Run this cell to download all necccesary libraries for spotdl to work
!apt-get remove ffmpeg
!add-apt-repository -y ppa:savoury1/ffmpeg4
!apt install -y ffmpeg
!pip install spotdl

In [ ]:
#@markdown ### Run cell to download playlist in colab
#@markdown #### Steps:
#@markdown 1. First enter playlist *url*
#@markdown 2.  select run after from Runtime or press ctrl+F10

#@markdown Note:- Make sure to run the reset cell in the end before running this cell again to download another playlist. 
#@markdown As it is essential that empty songs directory is created before the move command is executed.


#@title Enter spotify playlist URL

url = ' https://open.spotify.com/playlist/6an0hNyshVMWORG7qVNUbq ' #@param {type:"string"}


!spotdl url | grep -e [Found] > ~/sp.log
!mkdir songs
!find -type f -name "*.mp3" -exec mv {} /content/songs/ \;

In [ ]:
#@markdown ⬅️ Copy Song Folder To Drive 
DRIVE_PATH = "/content/drive/drive2" #@param {type:"string"}
!sudo cp -r -v "/content/songs/" "$DRIVE_PATH"

In [ ]:
#@markdown ### Zip the songs folder to download the folder


#@markdown **Note:- you can also transfer the zip file to your google drive by connecting it to the colab notebook. See this [resource](https://stackoverflow.com/questions/47744131/colaboratory-can-i-access-to-my-google-drive-folder-and-file) to know how to do it.** 
!zip -r /content/song.zip /content/songs

### Reset the colab environment to download another playlist 

### To do so :
1. The songs folder is deleted
2. The songs zip file is removed

In [ ]:
#@markdown ### Reset the colab environment to download another playlist 

#@markdown ### To do so :
#@markdown 1. The songs folder is deleted
#@markdown 2. The songs zip file is removed 

!rm -r /content/songs
!rm -r /content/songs.zip

In [ ]:
#@markdown ### Miscellaneous cells
# removing all mp3 files
!find -type f -name "*.mp3" -exec rm {} \;

# finding size of files
!ls -lh

# finding number of files in songs folder
!ls /content/songs | wc -l;


<font color='Red'><h1>One Google Drive To Other GDrive</font>

In [ ]:
#@markdown ###⬅️Mount Google Drive<br><h5>Frist Google Drive Which Include Share Drive
from google.colab import drive
drive.mount('/content/drive')

###MOUNT YOUR SECOND GOOGLE DRIVE 
$\color{red}{\text{Important Note}}$: THIS CELL WILL NOT ABLE MOUNT YOUR SHARE DRIVE

In [ ]:
#@title
#@markdown <h3>⬅️ Click Here to Mount Second Drive</h3> 

!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}
!sudo mkdir /content/drive1
!google-drive-ocamlfuse /content/drive1

In [ ]:
#@markdown ⬅️Transfer File/Folder One GDRIVE To Another GDRIVE
FILE_FOLDER_PATH = "/content/drive/mydrive" #@param {type:"string"}
NEW_PATH = "/content/drive/drive2" #@param {type:"string"}
!sudo cp -r -v "$FILE_FOLDER_PATH" "$NEW_PATH"

<font color='red'><h1>Mega Single/Multiple Links Downloader to Google Drive</font>

In [ ]:
#@markdown ### Mount Google Drive<br><h5>To stream files we need to mount Google Drive.
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# @markdown <font size=6 color="red">1. [Optional]</font><font size=5> Input Your Mega ID, Password (5GB+ Download )- if you have Pro/Business Account</font><br>
from functools import wraps
import errno
import os
import signal
import subprocess
import shlex


class TimeoutError(Exception):
    pass


def timeout(seconds=10, error_message=os.strerror(errno.ETIME)):
    def decorator(func):
        def _handle_timeout(signum, frame):
            raise TimeoutError(error_message)

        def wrapper(*args, **kwargs):
            signal.signal(signal.SIGALRM, _handle_timeout)
            signal.alarm(seconds)
            try:
                result = func(*args, **kwargs)
            finally:
                signal.alarm(0)
            return result

        return wraps(func)(wrapper)

    return decorator


if not os.path.exists("/root/.ipython/ocr.py"):
    from subprocess import run
    from shlex import split

    shellCmd = "wget -qq https://raw.githubusercontent.com/biplobsd/OneClickRun/master/res/ocr.py \
                    -O /root/.ipython/ocr.py"
    run(split(shellCmd))
from ocr import runSh

@timeout(10)
def runShT(args):
    return runSh(args, output=True)


# MEGAcmd installing
if not os.path.exists("/usr/bin/mega-cmd"):
    print("Installing MEGA ...")
    runSh('sudo apt-get -y update')
    runSh('sudo apt-get -y install libmms0 libc-ares2 libc6 libcrypto++6 libgcc1 libmediainfo0v5 libpcre3 libpcrecpp0v5 libssl1.1 libstdc++6 libzen0v5 zlib1g apt-transport-https')
    runSh('sudo curl -sL -o /var/cache/apt/archives/MEGAcmd.deb https://mega.nz/linux/MEGAsync/Debian_9.0/amd64/megacmd-Debian_9.0_amd64.deb', output=True)
    runSh('sudo dpkg -i /var/cache/apt/archives/MEGAcmd.deb', output=True)
    print("MEGA is installed.")
else:
    !pkill mega-cmd

# INPUT YOUR MEGA ID

#runShT(f"mega-logout")
USERNAME = "yourname@gmail.com"  # @param {type:"string"}
PASSWORD = "This ia a password"  # @param {type:"string"}
if not (USERNAME == "" or PASSWORD == ""):
    try:
        runShT(f"mega-login {USERNAME} {PASSWORD}")
    except TimeoutError:
        runSh('mega-whoami', output=True)
else:
    print("Please Input your Mega IDs.")
# @markdown *<font color="red">After signing in, use the below cell(Main Cell) to transfer files from Mega to any Cloud Drive using the transfer quota of your Mega Pro/Business Account*</font>

In [ ]:
#@title
import sys, os, urllib.request
import time
import subprocess
import contextlib
from IPython.display import clear_output
# @markdown <font size=6 color="red">2.</font><font size=5> MEGA Single/Multiple public links Downloader</font><br>
#@markdown <br><h2><b> Multiple URL's Supported
#@markdown <br><br><center><img src='https://mega.nz/favicon.ico?v=3' height="50" alt="MEGA-logo"/></center>
#@markdown <center><h2>Transfer from Single/Multiple Mega Links to any Cloud Drive</h2></center><br>
HOME = os.path.expanduser("~")
if not os.path.exists(f"{HOME}/.ipython/ocr.py"):
    hCode = "https://raw.githubusercontent.com/biplobsd/" \
                "OneClickRun/master/res/ocr.py"
    urllib.request.urlretrieve(hCode, f"{HOME}/.ipython/ocr.py")

from ocr import (
    runSh,
    loadingAn,
)
#@title
SINGLE_URL = "https://mega.nz/folder/euiKwguiue6ieZ#7wlzeuiycbGV-b5LvZeuu" #@param {type:"string"}
#BoostUpStation
MULTI_URL_FILE = False #@param {type:"boolean"}
MULTI_URL_FILE_PATH = "/content/link.txt" #@param {type:"string"}
#BoostUpStation
OUTPUT_PATH = "/content/drive/Shareddrives/" #@param {type:"string"}
#@markdown #####_<font color="red">*Sometimes this cell doesn't stop itself after the completion of the transfer. In case of that stop the cell manually._</font>

if not OUTPUT_PATH:
  os.makedirs("downloads", exist_ok=True)
  OUTPUT_PATH = "downloads"
# MEGAcmd installing
if not os.path.exists("/usr/bin/mega-cmd"):
    loadingAn()
    print("Installing MEGA ...")
    runSh('sudo apt-get -y update')
    runSh('sudo apt-get -y install libmms0 libc-ares2 libc6 libcrypto++6 libgcc1 libmediainfo0v5 libpcre3 libpcrecpp0v5 libssl1.1 libstdc++6 libzen0v5 zlib1g apt-transport-https')
    runSh('sudo curl -sL -o /var/cache/apt/archives/MEGAcmd.deb https://mega.nz/linux/MEGAsync/Debian_9.0/amd64/megacmd-Debian_9.0_amd64.deb', output=True)
    runSh('sudo dpkg -i /var/cache/apt/archives/MEGAcmd.deb', output=True)
    print("MEGA is installed.")
    clear_output()

# Unix, Windows and old Macintosh end-of-line
newlines = ['\n', '\r\n', '\r']

def unbuffered(proc, stream='stdout'):
    stream = getattr(proc, stream)
    with contextlib.closing(stream):
        while True:
            out = []
            last = stream.read(1)
            # Don't loop forever
            if last == '' and proc.poll() is not None:
                break
            while last not in newlines:
                # Don't loop forever
                if last == '' and proc.poll() is not None:
                    break
                out.append(last)
                last = stream.read(1)
            out = ''.join(out)
            yield out


def transfer():
    import codecs
    decoder = codecs.getincrementaldecoder("UTF-8")()
    #BoostUpStation
    def downloader(URL):
    #BoostUpStation
        cmd = ["mega-get", URL, OUTPUT_PATH]
        proc = subprocess.Popen(
            cmd,
            stdout=subprocess.PIPE,
            stderr=subprocess.STDOUT,
            # Make all end-of-lines '\n'
            universal_newlines=True,
        )
        for line in unbuffered(proc):
            print(line)
    #BoostUpStation
    if(MULTI_URL_FILE):
        with open(MULTI_URL_FILE_PATH) as f:
            Links = f.readlines()
            for LINK in Links:
                print("#############################")
                print("DOWNLOADING -- " + LINK)
                print("#############################")
                downloader(LINK)
    else:
        downloader(SINGLE_URL)
    #BoostUpStation
transfer()

<font color='red'><h1>Extract Zip/Tar/Rar/7z Files in Google Drive</font>

In [ ]:
#@markdown ###⬅️Mount Google Drive<br><h5>To stream files we need to mount Google Drive.

from google.colab import drive
drive.mount('/content/drive', force_remount=True)

In [ ]:
# ============================= FORM ============================= #
# @markdown #### ⬅️ Extract Files
MODE = "UNZIP"  # @param ["UNZIP", "UNTAR", "UNRAR", "7Z"]
PATH_TO_FILE = ""  # @param {type:"string"}
extractPath = ""  # @param {type:"string"}
ARCHIVE_PASSWORD = "" #@param {type:"string"}

# ================================================================ #
import os, urllib.request
HOME = os.path.expanduser("~")

if not os.path.exists(f"{HOME}/.ipython/ocr.py"):
    hCode = "https://raw.githubusercontent.com/biplobsd/" \
                "OneClickRun/master/res/ocr.py"
    urllib.request.urlretrieve(hCode, f"{HOME}/.ipython/ocr.py")

from ocr import (
    runSh,
    checkAvailable,
)

def extractFiles():
    global extractPath
    if ARCHIVE_PASSWORD:
      passADD = f'-p{ARCHIVE_PASSWORD}'
    else:
      passADD = ''
    if not extractPath:
      extractPath = "/content/extract"
    os.makedirs(extractPath, exist_ok=True)
    if MODE == "UNZIP":
        runSh('unzip '+passADD+f' "{PATH_TO_FILE}" -d "{extractPath}"', output=True)
    elif MODE == "UNRAR":
        runSh(f'unrar x "{PATH_TO_FILE}" "{extractPath}" '+passADD+' -o+', output=True)
    elif MODE == "UNTAR":
        runSh(f'tar -C "{extractPath}" -xvf "{PATH_TO_FILE}"', output=True)
    else:
        runSh(f'7z x "{PATH_TO_FILE}" -o{extractPath} '+passADD, output=True)


extractFiles()

<font color='red'><h1>Download Facebook Video</font>

* This code downloads a publicly available facebook video
* Just pass it the video link and where you'd like to save it on your machine

In [ ]:
#@markdown ⬅️ Import required packages
import sys
import os
import re
import requests as r
!pip install wget

In [ ]:
#@markdown  This is the function that does the download pass it the FB video URL and a path to the destination folder
def download_fbvid(link, dest_folder):
    try:
        html = r.get(link)
        hdvideo_url = re.search('hd_src:"(.+?)"', html.text)[1]
    except r.ConnectionError as e:
        print("OOPS!! Connection Error.")
        return False
    except r.Timeout as e:
        print("OOPS!! Timeout Error")
        return False
    except r.RequestException as e:
        print("OOPS!! General Error or Invalid URL")
        return False
    except (KeyboardInterrupt, SystemExit):
        print("Ok ok, quitting")
        sys.exit(1)
    except TypeError:
        print("Video May Private or Hd version not avilable")
        return False
    else:
        hd_url = hdvideo_url.replace('hd_src:"', '')
        wget.download(hd_url, dest_folder)
        print("Downloaded successful!")
        return True

In [ ]:
LINK = "" #@param {type:"string"}
DOWNLOAD_PATH = "/content/" #@param {type:"string"}
dest_folder = '$DOWNLOAD_PATH'
download_fbvid(LINK, dest_folder)

<font color='red'><h1> Compress Files/Folders to Zip/Tar/7z Archives in Google Drive</font>

In [ ]:
#@markdown ### Mount Google Drive<br><h5>To stream files we need to mount Google Drive.
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

In [ ]:
# ============================= FORM ============================= #
# @markdown #### ⬅️ Archive Files
MODE = "ZIP" #@param ["ZIP", "TAR", "7Z"]
FILENAME = "/content/drive/My Drive/password_protected.zip"  # @param {type:"string"}
PATH_TO_FILE = "/content/drive/My Drive/Subscribe/RAR for Android v5.91 build 95 [Final] [Premium] [Mod]"  # @param {type:"string"}
ARCHIVE_PASSWORD = "1234" #@param {type:"string"}

# option supports b k m g (bytes, kilobytes, megabytes, gigabytes)
SPLIT = "no" #@param ["1g", "2g", "3g", "4g", "5g", "no"]


compress = 6#@param  {type:"slider", min:0, max:9, step:0}
#@markdown <font size=2%>Use this to path separate between to this character |. Ex path/to /1 | path/to/2 </br>#Split only in zip mode.(Only for noobs)
# ================================================================ #
from pathlib import PurePosixPath

pathList = PATH_TO_FILE.split('|')
if MODE == "ZIP":
    if not FILENAME:
      FILENAME = "/content/NEW_FILE.ZIP"
    if ARCHIVE_PASSWORD:
      passADD = f'--password "{ARCHIVE_PASSWORD}"'
    else:
      passADD = ''
    splitC = f"-s {SPLIT}" if not 'no' in SPLIT else ""
    for part in pathList:
      pathdic = PurePosixPath(part.strip())
      parent = pathdic.parent
      partName = pathdic.parts[-1]
      cmd = f'cd "{parent}" && zip {passADD} -{compress} {splitC} -v -r -u "{FILENAME}" "{partName}"'
      !$cmd
elif MODE == "TAR":
    if not FILENAME:
      FILENAME = "/content/NEW_FILE"
    cmd = f'GZIP=-{compress} tar -zcvf "{FILENAME}.tar.gz" {PATH_TO_FILE}'
    !$cmd
else:
    if not FILENAME:
        FILENAME = "/content/NEW_FILE"
    for part in pathList:
      pathdic = PurePosixPath(part.strip())
      parent = pathdic.parent
      partName = pathdic.parts[-1]
      cmd = f'cd "{parent}" && 7z a -mx={compress} "{FILENAME}.7z" "{partName}"'
      !$cmd

<font color='red'><h1>Convert OR Compress Videos in Google Colab Google Drive</font>

In [ ]:
#@title ⬇️Install HandBrake and rClone
%%capture
AUTO_RECONNECT = True #@param {type:"boolean"}
HANDBRAKE = True #@param {type:"boolean"}
RCLONE = True #@param {type:"boolean"}
#@markdown Check AUTO_RECONNECT to prevent notebook from disconnecting!

from os import makedirs
makedirs("/content/temp/HandbrakeTemp", exist_ok = True)
makedirs("/root/.config/rclone", exist_ok = True)
if HANDBRAKE==True:
  !wget -qq https://github.com/vot/ffbinaries-prebuilt/releases/download/v4.2.1/ffmpeg-4.2.1-linux-64.zip 
  !rm -f ffmpeg-4.2.1-linux-64.zip
  !add-apt-repository ppa:stebbins/handbrake-releases -y 
  !apt-get install -y handbrake-cli 
  
if RCLONE==True:
  !curl https://rclone.org/install.sh | sudo bash

if AUTO_RECONNECT:
  import IPython
  from google.colab import output

  display(IPython.display.Javascript('''
  function ClickConnect(){
    btn = document.querySelector("colab-connect-button")
    if (btn != null){
      console.log("Click colab-connect-button");
      btn.click()
      }
    
    btn = document.getElementById('ok')
    if (btn != null){
      console.log("Click reconnect");
      btn.click()
      }
    }
    
  setInterval(ClickConnect,60000)
  '''))

In [ ]:
#@title <center> <strong>HandBrake Configuration</strong> </center>
#@markdown > Select Mode ( Batch conversion/ Single File)
MODE = "SINGLE" #@param ["SINGLE", "BATCH"]


#@markdown ---
SOURCE = "" #@param {type:"string"}
DESTINATION = "" #@param {type:"string"}
FORMAT = "mp4" #@param ["mp4", "mkv"]
RESOLUTION = "480p" #@param ["360p", "480p","540p", "720p", "1080p"]
Encoder = "x264" #@param ["x264", "x265"]
Encoder_Preset = "ultrafast" #@param ["ultrafast", "faster", "fast", "medium", "slow", "slower"]
#@markdown Choose Constant Quality Rate [Lower = Higher Quality/
#@markdown Larger File Size]
CQ = 30 #@param {type:"slider", min:10, max:30, step:1}
BURN_SUBTITLES = False #@param {type:"boolean"}
Additional_Flags = "" #@param {type:"string"}

########################################################
import smtplib
import os


formats = ('.mkv','.mp4','.ts','.avi','.mov','.wmv')

######## Renames the file ########
def fileName(fPath):
        tName = fPath.split('/')[-1]        
        if tName.endswith('ts'):
          tName = tName[:-3] + f' [{RESOLUTION}] [{Encoder}].{FORMAT}'    
        else:
          tName = tName[:-4] + f' [{RESOLUTION}] [{Encoder}].{FORMAT}'    
        return tName

def set_resolution():
  global w,h,flags
  if RESOLUTION == "360p":
    w, h = "480" , "360"
  elif RESOLUTION == "480p":
    w, h = "854" , "480"
  elif RESOLUTION == "540p":
    w, h = "960" , "540"
  elif RESOLUTION == "720p":
    w, h = "1280" , "720"
  elif RESOLUTION == "1080p":
    w, h = "1920" , "1080"

def addFlags():
  global flags
  flags = f" --encoder {Encoder}  --all-audio -s '0,1,2,3' --cfr --optimize --quality={CQ} --width={w} --height={h} --format={FORMAT} --encoder-preset={Encoder_Preset} "
  if BURN_SUBTITLES:
    flags += "-s '1' --subtitle-burned '1' "
  if Additional_Flags != "":
    flags += str(Additional_Flags)

set_resolution()
addFlags()

##### HandBrake and Rclone #####
def runner(path):
  f_name = fileName(path)
  hTemp=f"/content/temp/HandbrakeTemp/{f_name}"
  !HandBrakeCLI -i "$path" -o "$hTemp" $flags
  
      
  if os.path.isfile(hTemp):
    print(f"\n\n** Successfully converted {f_name}\n Now saving to Destination.....")
    if os.path.exists('/usr/bin/rclone'):
      !rclone move "$hTemp" --user-agent "Mozilla" "$DESTINATION" --transfers 20 --checkers 20 --stats-one-line --stats=5s -v --tpslimit 95 --tpslimit-burst 40
    else:
      dest = DESTINATION+'/'+f_name
      !mv "$hTemp" "$dest"
  if os.path.isfile(DESTINATION+ '/' +f_name):    
      print(f"\n\n** Successfully saved {f_name} to Destination")

########## Check Mode ########
if MODE=="BATCH":
  os.makedirs(DESTINATION, exist_ok=True)
  if SOURCE.endswith('/'):
      pass
  else: SOURCE +='/'
  filesList = os.listdir(SOURCE+'.')
  if os.path.isfile(SOURCE+'processed_db.txt'):
    pass
  else:
    with open((SOURCE+'processed_db.txt'), 'w') as fb:
      fb.write("Do not delete this file until all files have been processed!\n")
      fb.close()
  with open((SOURCE+'processed_db.txt'), "r+") as filehandle:
    processedList = [x.rstrip() for x in filehandle.readlines()]

    print('<<<<<<<<<<<<<<<<<< Starting Conversion in Batch mode. >>>>>>>>>>>>>>>>>>')

    for currentFile in filesList:
      if currentFile.endswith(formats):
        if currentFile not in processedList:
          currentPath = SOURCE + currentFile      
          print(f'\n\n**** Current File to process: {currentFile}')
          runner(currentPath)
          filehandle.write(currentFile+'\n')
    filehandle.close()
        

else:
    if SOURCE.endswith(formats):    
        runner(SOURCE)
    else: print("Are you sure you have selected the correct file??")

In [ ]:
#@markdown ### Mount Google Drive<br><h5>To stream files we need to mount Google Drive.
from google.colab import drive
drive.mount('/content/drive')

<font color='red'><h1>Mediafire to Google Drive</font>

In [ ]:
#@markdown ### Mount Google Drive<br><h5>To stream files we need to mount Google Drive.
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

In [ ]:
#@markdown ### Install Mediafire-dl and Initialize Session
!pip3 install git+https://github.com/Juvenal-Yescas/mediafire-dl

In [ ]:
#@markdown ##Download <h5>Enter your Download Link & Output file name(with type)
import mediafire_dl

url = 'https://www.mediafire.com/file/fg6ip510iw3zeyq/MediaFire_-_Getting_Started.pdf/file'#@param {type:"string"}
output = 'MediaFire_-_Getting_Started.pdf' #@param {type:"string"}
DOWNLOAD_PATH = "/content/drive/Mydrive" #@param {type:"string"}
mediafire_dl.download(url, output, quiet=False)
!sudo cp -r -v "/content/$output" "$DOWNLOAD_PATH"

<font color='red'><h1>Checking folders size in Google Drive</font>

In [ ]:
#@markdown ### Mount Google Drive To<br><h5> stream files we need to mount Google Drive.
from google.colab import drive drive.mount('/content/drive')

In [ ]:
#@markdown ###Enter Your Folder Path To find out the Disk usage
FOLDER_PATH = "/content/drive/Shareddrives/1" #@param {type:"string"}
!ls -l "$FOLDER_PATH"
!du -sh "$FOLDER_PATH"